In [9]:
from sentence_transformers import SentenceTransformer
import datetime
import json

import associative_memory
import perceive

### Exemplo

In [4]:
# Initialize the embeddings model
model = SentenceTransformer('all-MiniLM-L6-v2')

#### Sample nodes, kw_strength and embeddings

In [21]:
# Sample data for nodes.json
nodes_data ={
  "node_1": {
    "node_count": 1,
    "type_count": 1,
    "type": "event",
    "depth": 0,
    "created": "2024-06-01 08:00:00",
    "expiration": None,
    "subject": "Alex",
    "predicate": "attended",
    "object": "photography workshop",
    "description": "Alex attended a nature photography workshop in the city park.",
    "embedding_key": "embedding_1",
    "poignancy": 8,
    "keywords": ["photography", "workshop", "nature"],
    "filling": []
  },
  "node_2": {
    "node_count": 2,
    "type_count": 1,
    "type": "thought",
    "depth": 1,
    "created": "2024-06-01 19:00:00",
    "expiration": None,
    "subject": "Alex",
    "predicate": "thought",
    "object": "about new photography techniques",
    "description": "Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects.",
    "embedding_key": "embedding_2",
    "poignancy": 7,
    "keywords": ["photography", "learning", "techniques"],
    "filling": ["node_1"]
  },
  "node_3": {
    "node_count": 3,
    "type_count": 2,
    "type": "event",
    "depth": 0,
    "created": "2024-06-05 10:00:00",
    "expiration": None,
    "subject": "Alex",
    "predicate": "completed",
    "object": "software project",
    "description": "Alex completed a major software development project at work, delivering the final product to the client.",
    "embedding_key": "embedding_3",
    "poignancy": 9,
    "keywords": ["software", "project", "completion"],
    "filling": []
  },
  "node_4": {
    "node_count": 4,
    "type_count": 2,
    "type": "thought",
    "depth": 1,
    "created": "2024-06-05 20:00:00",
    "expiration": None,
    "subject": "Alex",
    "predicate": "felt",
    "object": "relief and accomplishment",
    "description": "After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.",
    "embedding_key": "embedding_4",
    "poignancy": 6,
    "keywords": ["relief", "accomplishment", "project"],
    "filling": ["node_3"]
  },
  "node_5": {
    "node_count": 5,
    "type_count": 3,
    "type": "event",
    "depth": 0,
    "created": "2024-06-10 16:00:00",
    "expiration": None,
    "subject": "Alex",
    "predicate": "explored",
    "object": "national park",
    "description": "Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes.",
    "embedding_key": "embedding_5",
    "poignancy": 8,
    "keywords": ["exploration", "photography", "nature", "trip"],
    "filling": []
  }
}

# Generate embeddings for the descriptions
embeddings_data = {
    "embedding_1": model.encode(nodes_data["node_1"]["description"]).tolist(),
    "embedding_2": model.encode(nodes_data["node_2"]["description"]).tolist(),
    "embedding_3": model.encode(nodes_data["node_3"]["description"]).tolist(),
    "embedding_4": model.encode(nodes_data["node_4"]["description"]).tolist(),
    "embedding_5": model.encode(nodes_data["node_5"]["description"]).tolist()
}

kw_strength_data = {
    "kw_strength_event": {
        "photography": 2,
        "workshop": 1,
        "nature": 2,
        "software": 1,
        "project": 1,
        "completion": 1,
        "exploration": 1,
        "trip": 1
    },
    "kw_strength_thought": {
        "photography": 1,
        "learning": 1,
        "techniques": 1,
        "relief": 1,
        "accomplishment": 1,
        "project": 1
    }
}

# Create initial files
with open("nodes.json", "w") as f:
    json.dump(nodes_data, f)
with open("embeddings.json", "w") as f:
    json.dump(embeddings_data, f)
with open("kw_strength.json", "w") as f:
    json.dump(kw_strength_data, f)

In [15]:
memory = associative_memory.AssociativeMemory(".")

#### Persona class (o próximo passa será implementar a versão completa da classe)

In [16]:
class Persona:
    def __init__(self):
        self.a_mem = memory
        self.scratch = type('', (), {})()
        self.scratch.recency_decay = 0.9
        self.scratch.recency_w = 1.0
        self.scratch.relevance_w = 1.0
        self.scratch.importance_w = 1.0
        self.scratch.curr_time = datetime.datetime.now()

In [17]:
persona = Persona()

#### Query to search for

In [18]:
# Define focal points, a question about the memory of Alex
focal_points = ["What did Alex think about after attending the photography workshop?"]

Abaixo, vemos os scores para *recency*, *relevance* e *importance*, todos os scores estão normalizados entre 0 e 1. Memórias acessadas recentemente tem um score de *recency* mais alto. O score de *importance* é obtido a partir do parâmetro *poignancy*. O score de *relevance* é obtido a partir da similaridade de cosseno entre a query e o embedding da descrição dos nodes.

In [19]:
retrieved_nodes = perceive.new_retrieve(persona, focal_points, debug=True)

Debug information for focal point: What did Alex think about after attending the photography workshop?

Node Creation Datetime and Recency Scores:
Node ID: node_1, Last Accessed: 2024-06-01 08:00:00, Recency Score: 0.0000
Node ID: node_2, Last Accessed: 2024-06-01 19:00:00, Recency Score: 0.2120
Node ID: node_3, Last Accessed: 2024-06-05 10:00:00, Recency Score: 0.4475
Node ID: node_4, Last Accessed: 2024-06-05 20:00:00, Recency Score: 0.7092
Node ID: node_5, Last Accessed: 2024-06-10 16:00:00, Recency Score: 1.0000

Importance Scores:
Node ID: node_1, Importance Score: 0.6667
Node ID: node_2, Importance Score: 0.3333
Node ID: node_3, Importance Score: 1.0000
Node ID: node_4, Importance Score: 0.0000
Node ID: node_5, Importance Score: 0.6667

Relevance Scores:
Node ID: node_1, Relevance Score: 1.0000
Node ID: node_2, Relevance Score: 0.0000
Node ID: node_3, Relevance Score: 0.4230
Node ID: node_4, Relevance Score: 0.7005
Node ID: node_5, Relevance Score: 0.5484

Node Scores Breakdown:
